In [1]:
import pandas as pd

from pathlib import Path

from datasets import load_dataset

/home/paul/.conda/envs/tf/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(


In [10]:
%matplotlib inline

In [3]:
root_dir = Path.cwd()
dataset = load_dataset("semeru/code-text-python", cache_dir=root_dir / "data" / "cache")

Found cached dataset json (/home/paul/projects/edu/master/mdl-ii/src/data/cache/semeru___json/semeru--code-text-python-047c4cae5928dad3/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
dataset.set_format(type="pandas")
train_df = dataset["train"][:]
test_df = dataset["test"][:]
validation_df = dataset["validation"][:]

# Data Understanding

In [5]:
df = pd.concat([train_df, test_df, validation_df])

In [13]:
df.head()

,repo,path,func_name,original_string,language,code,code_tokens,docstring,docstring_tokens,sha,url,partition
0,smdabdoub/phylotoast,phylotoast/util.py,split_phylogeny,"def split_phylogeny(p, level=""s""):\n """"""\n ...",python,"def split_phylogeny(p, level=""s""):\n """"""\n ...","[def, split_phylogeny, (, p, ,, level, =, ""s"",...",Return either the full or truncated version of...,"[Return, either, the, full, or, truncated, ver...",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0...,train
1,smdabdoub/phylotoast,phylotoast/util.py,ensure_dir,"def ensure_dir(d):\n """"""\n Check to make...",python,"def ensure_dir(d):\n """"""\n Check to make...","[def, ensure_dir, (, d, ), :, if, not, os, ., ...",Check to make sure the supplied directory path...,"[Check, to, make, sure, the, supplied, directo...",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0...,train
2,smdabdoub/phylotoast,phylotoast/util.py,file_handle,"def file_handle(fnh, mode=""rU""):\n """"""\n ...",python,"def file_handle(fnh, mode=""rU""):\n """"""\n ...","[def, file_handle, (, fnh, ,, mode, =, ""rU"", )...",Takes either a file path or an open file handl...,"[Takes, either, a, file, path, or, an, open, f...",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0...,train
3,smdabdoub/phylotoast,phylotoast/util.py,gather_categories,"def gather_categories(imap, header, categories...",python,"def gather_categories(imap, header, categories...","[def, gather_categories, (, imap, ,, header, ,...",Find the user specified categories in the map ...,"[Find, the, user, specified, categories, in, t...",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0...,train
4,smdabdoub/phylotoast,phylotoast/util.py,parse_unifrac,"def parse_unifrac(unifracFN):\n """"""\n Pa...",python,"def parse_unifrac(unifracFN):\n """"""\n Pa...","[def, parse_unifrac, (, unifracFN, ), :, with,...",Parses the unifrac results file into a diction...,"[Parses, the, unifrac, results, file, into, a,...",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0...,train


## Features

> * **repo**: the owner/repo
> * **path**: the full path to the original file
> * **func_name**: the function or method name
> * **original_string**: the raw string before tokenization or parsing
> * **language**: the programming language
> * **code**: the part of the original_string that is code
> * **code_tokens**: tokenized version of code
> * **docstring**: the top-level comment or docstring, if it exists in the original string
> * **docstring_tokens**: tokenized version of docstring
> * **sha**: this field is not being used [TODO: add note on where this comes from?]
> * **partition**: a flag indicating what partition this datum belongs to of {train, valid, test, etc.} This is not used by the model. Instead we rely on directory structure to denote the partition of the data.
> * **url**: the url for the code snippet including the line numbers

Quelle: [CodeSearchNet](https://github.com/github/CodeSearchNet#evaluation)

Bei dem verwendeten Datensatz handelt es sich um den CodeSearchNet Datensatz, welcher die oben beschriebenen Features <br>
enthält. Hierbei wurden an dem Datensatz vom SEMERU Lab folgende Anpassungen vorgenommen:


> * Remove examples that codes cannot be parsed into an abstract syntax tree.
> * Remove examples that #tokens of documents is < 3 or >256
> * Remove examples that documents contain special tokens (e.g. <img ...> or https:...)
> * Remove examples that documents are not English.

Quelle: [SEMERU Lab](https://huggingface.co/datasets/semeru/code-text-python)

### Feature: repo

Hierbei handelt es sich um den Github Pfad zum Repository. Dieser besteht aus einem Benutzername und dem Namen des <br>
Repository getrennt durch einen Schrägstrich. Das Feature enthält keine Null-Werte hat aber für die weitere Analyse <br>
keine Relevanz.

In [9]:
df["repo"].isna().values.any()

False

### Feature: 

### Feature: 

### Feature: 

### Feature: 

### Feature: 

### Feature: 

### Feature: 

### Feature: 

### Feature: 

### Feature: 

### Feature: 